<h1>ANALYSING THE DATASET</h1>

In [1]:
import pandas as pd
data = pd.read_csv("MalwareData.csv", sep="|")

In [2]:
data.head()

,Name,md5,Machine,SizeOfOptionalHeader,Characteristics,MajorLinkerVersion,MinorLinkerVersion,SizeOfCode,SizeOfInitializedData,SizeOfUninitializedData,...,ResourcesNb,ResourcesMeanEntropy,ResourcesMinEntropy,ResourcesMaxEntropy,ResourcesMeanSize,ResourcesMinSize,ResourcesMaxSize,LoadConfigurationSize,VersionInformationSize,legitimate
0,memtest.exe,631ea355665f28d4707448e442fbf5b8,332,224,258,9,0,361984,115712,0,...,4,3.262823,2.568844,3.537939,8797.000000,216,18032,0,16,1
1,ose.exe,9d10f99a6712e28f8acd5641e3a7ea6b,332,224,3330,9,0,130560,19968,0,...,2,4.250461,3.420744,5.080177,837.000000,518,1156,72,18,1
2,setup.exe,4d92f518527353c0db88a70fddcfd390,332,224,3330,9,0,517120,621568,0,...,11,4.426324,2.846449,5.271813,31102.272727,104,270376,72,18,1
3,DW20.EXE,a41e524f8d45f0074fd07805ff0c9b12,332,224,258,9,0,585728,369152,0,...,10,4.364291,2.669314,6.400720,1457.000000,90,4264,72,18,1
4,dwtrig20.exe,c87e561258f2f8650cef999bf643a731,332,224,258,9,0,294912,247296,0,...,2,4.306100,3.421598,5.190603,1074.500000,849,1300,72,18,1


In [3]:
legitimate = data[0:41323].drop(["legitimate"], axis=1)
malware = data[41323:].drop(["legitimate"], axis=1)

In [4]:
print("Shape of legitimate dataset is %s samples with %s features" %(legitimate.shape[0], legitimate.shape[1]))
print("Shape of malware dataset is %s samples with %s features" %(malware.shape[0], malware.shape[1]))

Shape of legitimate dataset is 41323 samples with 56 features
Shape of malware dataset is 96724 samples with 56 features


<h1>CLASSIFIER</h1>


In [5]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

In [6]:
data_input = data.drop(['Name', 'md5', 'legitimate', 'ResourcesMinSize', 'ResourcesMinEntropy', 'ResourcesMeanSize', 'ResourcesMaxSize', 'ResourcesMinSize', 'ResourcesMaxEntropy', 'ResourcesMinEntropy'], axis=1).values
labels = data["legitimate"].values
extratrees = ExtraTreesClassifier().fit(data_input, labels)
select = SelectFromModel(extratrees, prefit=True)
data_input_new = select.transform(data_input)

In [7]:
data_input_new

array([[3.32000000e+02, 2.24000000e+02, 2.58000000e+02, ...,
        3.60742958e+00, 7.22105073e+00, 1.60000000e+01],
       [3.32000000e+02, 2.24000000e+02, 3.33000000e+03, ...,
        2.37352510e+00, 6.56690933e+00, 1.80000000e+01],
       [3.32000000e+02, 2.24000000e+02, 3.33000000e+03, ...,
        4.88519107e+00, 7.60095678e+00, 1.80000000e+01],
       ...,
       [3.32000000e+02, 2.24000000e+02, 2.58000000e+02, ...,
        4.21793766e+00, 7.97742342e+00, 1.40000000e+01],
       [3.32000000e+02, 2.24000000e+02, 3.31660000e+04, ...,
        0.00000000e+00, 6.43118768e+00, 0.00000000e+00],
       [3.32000000e+02, 2.24000000e+02, 2.58000000e+02, ...,
        3.90701074e+00, 6.56274547e+00, 0.00000000e+00]])

In [8]:
print(data_input.shape, data_input_new.shape)

(138047, 49) (138047, 11)


In [9]:
import numpy as np
features = data_input_new.shape[1]
importances = extratrees.feature_importances_
indices = np.argsort(importances)[::-1]

<h1>SELECTING THE MOST IMPORTANT FEATURES</h1>

In [10]:
for x in range(features):
    print("%d"%(x+1), data.columns[2+indices[x]], importances[indices[x]])

1 DllCharacteristics 0.17481647901907327
2 Machine 0.12955695406135473
3 Characteristics 0.12216155893942425
4 ResourcesMaxEntropy 0.07426482655691617
5 Subsystem 0.06571253593815855
6 ImageBase 0.05801063675652847
7 SectionsMaxEntropy 0.05496240397779197
8 MajorSubsystemVersion 0.04652667674227959
9 SizeOfOptionalHeader 0.0343861936345379
10 MajorOperatingSystemVersion 0.023967459007457753
11 SectionsMinEntropy 0.0222213208876224


<h1>RANDOM FOREST CLASSIFIER</h1>

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
legitimate_train, legitimate_test, malware_train, malware_test = train_test_split(data_input_new, labels, test_size=0.2)

In [12]:
classifier = RandomForestClassifier(n_estimators=100)

In [13]:
classifier.fit(legitimate_train, malware_train)

RandomForestClassifier()

In [14]:
print("Algorithm's score : ", classifier.score(legitimate_test, malware_test)*100)

Algorithm's score :  99.29735603042376


In [15]:
from sklearn.metrics import confusion_matrix
result = classifier.predict(legitimate_test)
matrix = confusion_matrix(malware_test, result)

In [16]:
matrix

array([[19244,   116],
       [   78,  8172]], dtype=int64)

In [17]:
print("False Positives : ", matrix[0][1]/sum(matrix[0])*100)
print("False Negatives : ", matrix[1][0]/sum(matrix[1])*100)

False Positives :  0.5991735537190083
False Negatives :  0.9454545454545455


In [18]:
import pickle

In [19]:
with open('model','wb') as file:
    pickle.dump(classifier, file)

In [20]:
import pickle
model_load = pickle.load(open('model','rb'))

In [21]:
import os
import pefile
import pandas as pd

def extract(path):
    pe = pefile.PE(path, fast_load=True)
    data = []
    entropy = list(map(lambda x:x.get_entropy(), pe.sections))
    SectionsMinEntropy = min(entropy)
    SectionsMaxEntropy = max(entropy)
    data.append(pe.OPTIONAL_HEADER.DllCharacteristics)
    data.append(pe.FILE_HEADER.Characteristics)
    data.append(pe.FILE_HEADER.Machine)
    data.append(pe.OPTIONAL_HEADER.ImageBase)
    data.append('6.9')
    data.append(SectionsMaxEntropy) 
    data.append(pe.OPTIONAL_HEADER.Subsystem)
    data.append(pe.FILE_HEADER.SizeOfOptionalHeader)
    data.append(pe.OPTIONAL_HEADER.MajorSubsystemVersion)
    data.append(SectionsMinEntropy)    
    data.append(pe.OPTIONAL_HEADER.MajorOperatingSystemVersion)
    data2 = []
    data2.append(data)
    return data2

attributes = extract('')
df = pd.DataFrame(attributes)
print(df.shape)

FileNotFoundError: [WinError 3] The system cannot find the path specified: ''

In [ ]:
df1 = pd.DataFrame(data_input_new[4])
df1 = np.transpose(df1)
#print(df1.shape)
prediction = model_load.predict(df1)
print(prediction)

: 

In [ ]:
x=np.argmax(prediction.round(), axis=0)
if(x==0):
    print("LEGITIMATE")
else:
    print("MALWARE")

: 

<h1>GRADIENT BOOST CLASSIFIER</h1>

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
grad_boost = GradientBoostingClassifier(n_estimators=50)
grad_boost.fit(legitimate_train, malware_train)

: 

In [ ]:
print("Score of Gradient Boost Classifier : ", grad_boost.score(legitimate_test, malware_test)*100)

: 

In [ ]:
from sklearn.metrics import confusion_matrix
result = grad_boost.predict(legitimate_test)
matrix = confusion_matrix(malware_test, result)

: 

In [ ]:
print("False Positives : ", matrix[0][1]/sum(matrix[0])*100)
print("False Negatives : ", matrix[1][0]/sum(matrix[1])*100)

: 

In [ ]:
import pickle
with open('Grad_Boost_model','wb') as file:
    pickle.dump(grad_boost, file)

: 

In [ ]:
import os
import pefile
import pandas as pd

def extract(path):
    pe = pefile.PE(path, fast_load=True)
    data = []
    entropy = list(map(lambda x:x.get_entropy(), pe.sections))
    SectionsMinEntropy = min(entropy)
    SectionsMaxEntropy = max(entropy)
    data.append(pe.OPTIONAL_HEADER.DllCharacteristics)
    data.append(pe.FILE_HEADER.Characteristics)
    data.append(pe.FILE_HEADER.Machine)
    data.append(pe.OPTIONAL_HEADER.ImageBase)
    data.append('6.9')
    data.append(SectionsMaxEntropy) 
    data.append(pe.OPTIONAL_HEADER.Subsystem)
    data.append(pe.FILE_HEADER.SizeOfOptionalHeader)
    data.append(pe.OPTIONAL_HEADER.MajorSubsystemVersion)
    data.append(SectionsMinEntropy)    
    data.append(pe.OPTIONAL_HEADER.MajorOperatingSystemVersion)
    data2 = []
    data2.append(data)
    return data2

attributes = extract('')
df = pd.DataFrame(attributes)
print(df.shape)

: 

In [ ]:
df1 = pd.DataFrame(data_input_new[4])
df1 = np.transpose(df1)
#print(df1.shape)
prediction = grad_boost.predict(df1)
print(prediction)
x=np.argmax(prediction.round(), axis=0)
if(x==0):
    print("LEGITIMATE")
else:
    print("MALWARE")

: 

<h1>TESTING OTHER CLASSIFIERS</h1>

In [ ]:
from sklearn import tree, linear_model
import sklearn.ensemble as ek
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
model = { "DecisionTree":tree.DecisionTreeClassifier(max_depth=10),
         "RandomForest":ek.RandomForestClassifier(n_estimators=50),
         "Adaboost":ek.AdaBoostClassifier(n_estimators=50),
         "GradientBoosting":ek.GradientBoostingClassifier(n_estimators=50),
         "GNB":GaussianNB(),
         "LinearRegression":LinearRegression()   
        }

: 

In [ ]:
for x in model:
    classifier = model[x]
    classifier.fit(legitimate_train, malware_train)
    score = classifier.score(legitimate_test, malware_test)
    print(x+ ' '+ str(score))

: 

: 